In [2]:
print("Hiiiiiiii")

Hiiiiiiii


In [10]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from pinecone import Pinecone, ServerlessSpec
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

from dotenv import load_dotenv 
import os

In [11]:
load_dotenv()
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_API_ENV = os.getenv('PINECONE_API_ENV')

In [12]:
# Extract data from the pdf 
def load_pdf(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [13]:

extracted_data=load_pdf("data/")

In [14]:
# create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [15]:

text_chunks = text_split(extracted_data)
print(len(text_chunks))

7020


In [16]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [17]:
embeddings = download_hugging_face_embeddings()

/tmp/ipykernel_247445/4238859041.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [18]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [19]:
query_result = embeddings.embed_query("Hello world")
print(len(query_result))

384


In [21]:
index_name = "medical-chatbot2"

pc=Pinecone(api_key=PINECONE_API_KEY)

index = pc.Index(index_name)

pc.list_indexes()

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 384,
              'host': 'medical-chatbot2-o7nzv1n.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'medical-chatbot2',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [22]:
def upload_to_pinecone(chunks, batch_size, embeddings):
    try:
        texts = [chunk.page_content for chunk in chunks]
        metadata = [{"source": chunk.metadata.get("source", ""),
                    "page": chunk.metadata.get("page", 0)} for chunk in chunks]
        
        embeddings = embeddings.embed_documents(texts)
        
        # Prepare vectors with metadata
        vectors = [
            (str(i), emb, meta) 
            for i, (emb, meta) in enumerate(zip(embeddings, metadata))
        ]
        
        # Upload in batches
        for i in range(0, len(vectors), batch_size):
            batch = vectors[i:i + batch_size]
            index.upsert(vectors=[(id_, vec, meta) for id_, vec, meta in batch])
            print(f"Uploaded batch {i//batch_size + 1} of {len(vectors)//batch_size + 1}")
            
    except Exception as e:
        print(f"Error uploading to Pinecone: {str(e)}")
        raise

In [78]:
upload_to_pinecone(text_chunks, 1000, embeddings)

Uploaded batch 1 of 8
Uploaded batch 2 of 8
Uploaded batch 3 of 8
Uploaded batch 4 of 8
Uploaded batch 5 of 8
Uploaded batch 6 of 8
Uploaded batch 7 of 8
Uploaded batch 8 of 8


In [23]:

texts = [chunk.page_content for chunk in text_chunks]
metadatas = [chunk.metadata for chunk in text_chunks]



In [24]:
from langchain.vectorstores import Pinecone as LangChainPinecone
docsearch = LangChainPinecone.from_texts(
    texts=texts,
    embedding=embeddings,
    metadatas=metadatas,
    index_name=index_name,
    namespace=""  # Optional: specify namespace if needed
)

In [25]:
def query_documents(query: str, k: int = 3):
        try:
            results = docsearch.similarity_search_with_score(query,k=k)
            return results
        except Exception as e:
            print(f"Error querying documents: {str(e)}")

In [27]:
query = "schizophrenia disorder"
results = query_documents(query, 6)
    
for doc, score in results:
    print(f"Score: {score}")
    print(f"Content: {doc.page_content[:600]}...")
    print("-" * 80)

Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.


Score: 0.669756114
Content: Psychosis —A serious mental disorder character-
ized by defective or lost contact with reality oftenwith hallucinations or delusions.
• aggitation
• memory loss
• confusion• dizziness
• tiredness
• headache• sleep disturbances
• stuttering
• dry skin• nausea
• constipation
•f e v e r• weight gain
• visual disturbances
Interactions
Taking atypical antipsychotic medications with cer-...
--------------------------------------------------------------------------------
Score: 0.669322431
Content: Psychosis —A serious mental disorder character-
ized by defective or lost contact with reality oftenwith hallucinations or delusions.
• aggitation
• memory loss
• confusion• dizziness
• tiredness
• headache• sleep disturbances
• stuttering
• dry skin• nausea
• constipation
•f e v e r• weight gain
• visual disturbances
Interactions
Taking atypical antipsychotic medications with cer-...
--------------------------------------------------------------------------------
Score:

In [28]:
prompt_template="""
Use the following peices of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else
Helpful answer:
"""

PROMPT= PromptTemplate(template=prompt_template, input_variables=["context", "question"] )
chain_type_kwargs= {"prompt": PROMPT}

In [29]:
llm= CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_1.bin",
                   model_type="llama",
                   config={'max_new_tokens': 512,
                            'temperature': 0.8})

In [30]:
qa = RetrievalQA.from_chain_type(
    llm= llm,
    chain_type= "stuff",
    retriever=docsearch.as_retriever(search_kwargs={'k':2}),
    return_source_documents= True,
    chain_type_kwargs= chain_type_kwargs
)

In [34]:
user_input="how to treat cold"
result=qa({"query": user_input})
print("Response: ", result["result"])

Found document with no `text` key. Skipping.


Response:  You can take over-the-counter pain relievers such as acetaminophen or ibuprofen to help reduce any discomfort or fever associated with a cold. Drinking plenty of fluids, getting enough rest, and using a humidifier to add moisture to the air can also help relieve symptoms. If your symptoms are severe or last longer than a week, you should consult a doctor for further treatment.
